<h1> Uygulamalı Veri Bilimi Ödevi Veri Analiz Bölümü </h1>

<h3>İsim: Vehbi Özcan</h3>
<h3>Numara: 211220070</h3>

## Import Bölümü

In [ ]:
import pandas as pd
import numpy as np
#Preprocessing işlemleri için
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
#Eğitim verisini parçalamak için 
from sklearn.model_selection import train_test_split
#Model Kayıt İşlemleri ve Model Yüklemek İçin Joblib
from joblib import Parallel, delayed 
import joblib 
#XGBoost Classifier
import xgboost as xgb
#Accuracy Score Metrics
from sklearn.metrics import accuracy_score



## Veriyi Okuma

In [ ]:
org_data_ = pd.read_excel('data.xlsx')
#data = org_data.copy()
#org_data.head()


In [ ]:
df_ = org_data_.copy()
df_.head()

## Preprocessing

In [ ]:
# df["Bellek Hızı"].fillna(value="3200 MHz", inplace=True)
# df["Cihaz Ağırlığı"].fillna(value="1 - 2 kg", inplace=True)
# df["Ekran Boyutu"].fillna(value="15,6 inç", inplace=True)
# df["Ekran Kartı Bellek Tipi"].fillna(value="Onboard", inplace=True)
# df["Ekran Kartı Hafızası"].fillna(value="Paylaşımlı", inplace=True)
# df["Ekran Kartı Tipi"].fillna(value="Dahili Ekran Kartı", inplace=True)
# df["Ekran Panel Tipi"].fillna(value="IPS", inplace=True)
# df["İşlemci Nesli"].fillna(value="11.Nesil", inplace=True)
# df["İşlemci"].fillna(value="1135G7", inplace=True)
# df["İşletim Sistemi"].fillna(value="Yok (Free Dos)", inplace=True)
# df["Kart Okuyucu"].fillna(value="Var", inplace=True)
# df["Aydınlatma"].fillna(value=0, inplace=True)
# df["Maksimum İşlemci Hızı"].fillna(value="4,7 GHz", inplace=True)
# df["Max Ekran Çözünürlüğü"].fillna(value="1920 x 1080", inplace=True)
# df["Parmak İzi Okuyucu"].fillna(value="Yok", inplace=True)
# df["Ram (Sistem Belleği)"].fillna(value="16 GB", inplace=True)
# df["SSD Kapasitesi"].fillna(value="512 GB", inplace=True)
# df["Ürün Modeli"].fillna(value="Notebook", inplace=True)
# df["Fiyat"].fillna(value="Düşük", inplace=True)



### Boş Verileri Doldurma Fonksiyonu

In [ ]:
def fill_value(df):
    for column in df.columns:
        df[column].fillna(value = df[column].mode()[0], inplace=True)
    return df

### Aykırı ve Gürültülü Verileri Düzeltme Fonksiyonu


In [ ]:
def fix_value(df):
    df['İşlemci'].replace('10. Nesil Intel® Core™ i3-10110U işlemci', '10110U',inplace=True)
    df['İşlemci Nesli'].replace('10. Nesil', '10.Nesil',inplace=True)
    df['SSD Kapasitesi'].replace(' 0 GB', '0 GB',inplace=True)
    df['Ram (Sistem Belleği)'].replace('8\xa0GB', '8 GB',inplace=True)
    return df

### Normalizasyon (MinMaxScaler) Fonksiyonu

In [ ]:
def df_minmaxscaler(le_df):
    scaler = MinMaxScaler()
    le_dfr = np.reshape(le_df,(-1,1))
    le_dfr = le_dfr.astype(float)
    le_dfs = scaler.fit_transform(le_dfr)
    le_dfs = le_dfs.flatten()
    return le_dfs

### Encoding (One Hot Encoder ve LabelEncoder) Fonksiyonu

In [ ]:
def df_encode(df):
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform='pandas')
    le = LabelEncoder()
    columns = df.columns.to_list()
    columns.remove("Fiyat")
    encoded_df = ohe.fit_transform(df[columns])
    le_df = le.fit_transform(df["Fiyat"])
    #le_dfs = df_minmaxscaler(le_df)
    #encoded_df["Fiyat"] = le_dfs
    encoded_df["Fiyat"] = le_df
    return encoded_df

## Preprocessing Function

In [ ]:
def preprocess(df):
    df = fill_value(df)
    df = fix_value(df)
    encoded_df = df_encode(df)
    return encoded_df

In [ ]:
# encoded_df = preprocess(df)
# encoded_df


## Classification Function

In [ ]:
def classification():
    org_data = pd.read_excel('data.xlsx')
    df = org_data.copy()

    encoded_df = preprocess(df)

    X = encoded_df.drop(["Fiyat"],axis=1) 
    y = encoded_df["Fiyat"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 60)

    xgb_cls = xgb.XGBClassifier(objective='multiclass:softmax', num_class=5)
    xgb_model = xgb_cls.fit(X_train,y_train)
    y_pred = xgb_model.predict(X_test)

    joblib.dump(xgb_model, 'modelxgb.pkl')
    score = accuracy_score(y_test,y_pred)
    print(score)
    return score



In [ ]:
classification()

## Test Function

In [ ]:
def test():
    test_data = pd.read_excel('data.xlsx')
    test_df = test_data.copy()

    encoded_df = preprocess(test_df)

    X_test = encoded_df.drop(["Fiyat"],axis=1) 
    y_test = encoded_df["Fiyat"]

    xgb_model = joblib.load('modelxgb.pkl') 
    y_pred = xgb_model.predict(X_test)
    score = accuracy_score(y_test,y_pred)
    print(score)
    return score


In [ ]:
test()